In [1]:
import sympy as sym
import helpers
from IPython.display import display
from IPython.display import Code

## ST-VK Hyperelastic Model

In [2]:
# Define input variables and constants.

f00, f01, f02, f10, f11, f12, f20, f21, f22 = f = sym.symbols('f00 f01 f02 f10 f11 f12 f20 f21 f22')
lame1, lame2 = sym.symbols('lambda mu', constants = True)

# Define expression to evaluate.

F = sym.Matrix([[ f00, f01, f02 ], [ f10, f11, f12 ], [ f20, f21, f22 ]])
I = sym.eye(3)
E1 = (F.T * F - I) / 2
E2 = E1 * E1

trE1 = sym.Trace(E1).doit() # Explicitly evaluate trace. Sympy has problems differentiating
trE2 = sym.Trace(E2).doit() # it otherwise.

U = lame1 * (trE1 ** 2) / 2 + lame2 * trE2
dUdF = helpers.gradient(U, f)
d2UdF2 = helpers.hessian(U, f)

# Define specific routines to export to code.

export = [("U", U), ("dUdF", dUdF), ("d2UdF2", d2UdF2)]

# Generate code and save to disk.

code = helpers.generate(export)
helpers.save('generated/stvk-U.c', code[0])
helpers.save('generated/stvk-dUdF.c', code[1])
helpers.save('generated/stvk-d2UdF2.c', code[2])

# Display.

#print('Energy')
#display(U)
#print('Gradient')
#display(dUdF)
#print('Hessian')
#display(d2UdF2)


## ST-VK Dissipation Potential

In [3]:
# Define input variables and constants.

f00, f01, f02, f10, f11, f12, f20, f21, f22 = f = sym.symbols('f00 f01 f02 f10 f11 f12 f20 f21 f22')
g00, g01, g02, g10, g11, g12, g20, g21, g22 = g = sym.symbols('g00 g01 g02 g10 g11 g12 g20 g21 g22')
lame1, lame2 = sym.symbols('lambda mu', constants = True)

# Define expression to evaluate.

F = sym.Matrix([[ f00, f01, f02 ], [ f10, f11, f12 ], [ f20, f21, f22 ]])
G = sym.Matrix([[ g00, g01, g02 ], [ g10, g11, g12 ], [ g20, g21, g22 ]])
I = sym.eye(3)
E1 = (G.T * F + F.T * G) / 2
E2 = E1 * E1

trE1 = sym.Trace(E1).doit()
trE2 = sym.Trace(E2).doit()

R = lame1 * (trE1 ** 2) / 2 + lame2 * trE2
dRdFG = helpers.gradient(R, [ *f, *g ])
d2RdFG2 = helpers.hessian(R, [ *f, *g ])

# Define specific routines to export to code.

export = [("R", R), ("dRdFG", dRdFG), ("d2RdF2", d2RdFG2)]


# Generate code and save to disk.

code = helpers.generate(export)
helpers.save('generated/stvk-dissip-R.inc', code[0])
helpers.save('generated/stvk-dissip-dRdFG.inc', code[1])
helpers.save('generated/stvk-dissip-d2RdFG2.inc', code[2])

# Display.

#print('Dissipation potential')
#display(R)
#print('Gradient')
#display(dRdFG)
#print('Hessian')
#display(d2RdFG2)
#print('Code')
